<a href="https://colab.research.google.com/github/RV05/Miscellaneous-tasks-DL/blob/main/densenet_disaster_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/deep learning

/content/drive/MyDrive/deep learning


In [3]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
import config
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19


from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import GlobalAveragePooling2D

from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
# ap = argparse.ArgumentParser()
# ap.add_argument("-p", "--plot", type=str, default="plot.png",
# 	help="path to output loss/accuracy plot")
# args = vars(ap.parse_args())
# # determine the total number of image paths in training, validation,
# # and testing directories


In [4]:
totalTrain = len(list(paths.list_images(config.TRAIN_PATH)))
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))
trainAug = ImageDataGenerator(
	# rotation_range=25,
	# zoom_range=0.1,
	# width_shift_range=0.1,
	# height_shift_range=0.1,
	# shear_range=0.2,
	# horizontal_flip=True,
  # vertical_flip=True,
	# fill_mode="nearest"
  )

In [5]:
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

In [6]:
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
# mean = np.array([123.68, 116.779, 103.939], dtype="float32")
# trainAug.mean = mean
# valAug.mean = mean

In [7]:
trainGen = trainAug.flow_from_directory(
	config.TRAIN_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=True,
	batch_size=config.BS)
# initialize the validation generator
valGen = valAug.flow_from_directory(
	config.VAL_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)
# initialize the testing generator
testGen = valAug.flow_from_directory(
	config.TEST_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)

Found 3309 images belonging to 6 classes.
Found 31 images belonging to 6 classes.
Found 1022 images belonging to 6 classes.


In [8]:
print("[INFO] preparing model...")
baseModel = DenseNet121(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(200, 200, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D()(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)

headModel = Dropout(0.5)(headModel)
headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False
# compile the model
opt = Adam(lr=config.INIT_LR, decay=config.INIT_LR / config.NUM_EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the model

[INFO] preparing model...
29097984/29084464 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [9]:

# load the ResNet-50 network, ensuring the head FC layer sets are left
# off

print("[INFO] training model...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // config.BS,
	validation_data=valGen,
	validation_steps=totalVal // config.BS,
	epochs=config.NUM_EPOCHS)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


[INFO] training model...
Epoch 1/200
413/413 [==============================] - 1656s 4s/step - loss: 1.4657 - accuracy: 0.4680 - val_loss: 1.5646 - val_accuracy: 0.3333
Epoch 2/200
413/413 [==============================] - 29s 70ms/step - loss: 1.0136 - accuracy: 0.6247 - val_loss: 1.5157 - val_accuracy: 0.4583
Epoch 3/200
413/413 [==============================] - 29s 70ms/step - loss: 0.8175 - accuracy: 0.7037 - val_loss: 1.6161 - val_accuracy: 0.4167
Epoch 4/200
413/413 [==============================] - 29s 69ms/step - loss: 0.6779 - accuracy: 0.7601 - val_loss: 1.8403 - val_accuracy: 0.5000
Epoch 5/200
413/413 [==============================] - 28s 68ms/step - loss: 0.5634 - accuracy: 0.7964 - val_loss: 1.6789 - val_accuracy: 0.4583
Epoch 6/200
413/413 [==============================] - 29s 69ms/step - loss: 0.4724 - accuracy: 0.8261 - val_loss: 1.7074 - val_accuracy: 0.5000
Epoch 7/200
413/413 [==============================] - 29s 69ms/step - loss: 0.3759 - accuracy: 0.8685 - 

In [10]:
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // config.BS) + 1)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
# serialize the model to disk
print("[INFO] saving model...")
model.save(config.MODEL_PATH, save_format="h5")
N = config.NUM_EPOCHS

[INFO] evaluating network...


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


              precision    recall  f1-score   support

        fire       0.78      0.78      0.78       200
       human       0.55      0.64      0.59        42
       infra       0.41      0.69      0.51       210
        land       0.53      0.44      0.48       148
    nodamage       0.49      0.11      0.18       211
       water       0.59      0.70      0.64       211

    accuracy                           0.55      1022
   macro avg       0.56      0.56      0.53      1022
weighted avg       0.56      0.55      0.52      1022

[INFO] saving model...


In [11]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("densenet disaster_Adam_plot.png")

In [12]:
H.history

{'accuracy': [0.46803998947143555,
  0.6246591806411743,
  0.7037261724472046,
  0.7600727081298828,
  0.7964253425598145,
  0.8261132836341858,
  0.8685246706008911,
  0.8857921957969666,
  0.9215389490127563,
  0.9294153451919556,
  0.9382005333900452,
  0.9578915238380432,
  0.9478945732116699,
  0.9663738012313843,
  0.9418358206748962,
  0.9615268111228943,
  0.9794001579284668,
  0.9709178805351257,
  0.9651620984077454,
  0.9812178015708923,
  0.9742501974105835,
  0.9797031283378601,
  0.9642532467842102,
  0.9818236827850342,
  0.9839442372322083,
  0.9660708904266357,
  0.9869736433029175,
  0.9806119203567505,
  0.9869736433029175,
  0.971220850944519,
  0.9890941977500916,
  0.9706149697303772,
  0.9772796034812927,
  0.9875795245170593,
  0.9890941977500916,
  0.9806119203567505,
  0.9724326133728027,
  0.9827325344085693,
  0.988791286945343,
  0.9781884551048279,
  0.9851559996604919,
  0.9806119203567505,
  0.9915177226066589,
  0.9933353662490845,
  0.9890941977500916,